In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [ ]:
if summit:
    print(os.environ["OSPL_URI"])
    print(os.environ["LSST_DDS_PARTITION_PREFIX"])
    print(os.environ["LSST_DDS_DOMAIN_ID"])
else:
    import os
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
#await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
m2 = salobj.Remote(script.domain, "MTM2")
print(f'time to start is {datetime.now() - start_time} [s]')

await m2.start_task

In [ ]:
#As long as you get something for the payload its OK. we dont' care about h.heartbeat
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
state = await m2.evt_summaryState.aget(timeout=5)
print('starting m2 state', state.summaryState,pd.to_datetime(state.private_sndStamp, unit='s'))
if not state.summaryState == 2:
    await salobj.set_summary_state(m2, salobj.State.ENABLED) #enable m2
    await asyncio.sleep(3)
    state = await m2.evt_summaryState.aget(timeout=5)
    print('m2 state', state.summaryState,pd.to_datetime(state.private_sndStamp, unit='s'))

In [ ]:
fa = [0]*72
ft = [0]*6
await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)

In [ ]:
holdTime = 5 #the M2 simulator publishes telemetry at 1Hz
fa = [0]*72
ft = [0]*6
start_time = datetime.now()
print('start time: ', start_time)
for i in range(72):
    fa[i] = 20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print('axial %d: +'%i, end='')
    await asyncio.sleep(holdTime)
    fa[i] = -20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' -', end='')
    await asyncio.sleep(holdTime)
    fa[i] = 0
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' 0 ', end='')
    await asyncio.sleep(holdTime)
    
for i in range(6):
    ft[i] = 20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print('tangent A%d: +'%i, end='')
    await asyncio.sleep(holdTime)
    ft[i] = -20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' -', end='')
    await asyncio.sleep(holdTime)
    ft[i] = 0
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' 0 ', end='')
    await asyncio.sleep(holdTime)    
print(f'\n time elapsed: {datetime.now() - start_time} [s]')

In [ ]:
#if we started with disabled state, we need to put it back
await salobj.set_summary_state(m2, salobj.State.OFFLINE) 